In [1]:
GRID_SIZE = 10
GRID_UNITINIALIZED = -1
GRID_UNINITIALIZED_SYMBOL = ' '
GRID_WALL = GRID_SIZE**2+1
GRID_WALL_SYMBOL = 'W'
GRID_PATH = 'x'
GRID_STATION = 'S'

In [2]:
def plot_grid(grid, message = None, positions = None):
    if positions:
        for p in positions:
            y,x = p
            grid[y][x] = GRID_STATION
    if message:
        print('\n'+message)
    for row in grid:
        res = '|'
        res += '\t| '.join([str(cell).replace(str(GRID_WALL),GRID_WALL_SYMBOL).replace(str(GRID_UNITINIALIZED),GRID_UNINITIALIZED_SYMBOL) for cell in row])
        res += '\t|'
        print(res)            

In [3]:
def solve_grid(stations, walls = None, route = None, depth = None):   
    # Initialize Variables
    depth = depth or 0
    walls = walls or []
    route = route or []
    grid_point_from = stations[-(2+depth)] # Start with the last two points and work your way to the front
    grid_point_to = stations[-(1+depth)]
    
    # Generate the empty grid
    grid = [[GRID_UNITINIALIZED for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    
    # Mark the known wall-positions
    for w_pos in walls:
        y,x = w_pos
        grid[y][x] = GRID_WALL
    
    # For Debugging
    plot_grid(grid,f'Created grid with walls, depth {depth}')
    
    # Calculate gradient-field to grid_point_from for backtracking
    walked = 0 # Pointer to remember how far we have traveled from the grid_target
    points_to_inspect = [[grid_point_from]] # List to hold batches of points with same distance
    while len(points_to_inspect) > 0: # If there still are points to inspect
        new_batch = [] # List to hold new points to inspect which have walkted-distance+=1
        batch = points_to_inspect.pop(0) # Get the 0-index item and remove it from list
        for pos in batch: # Iterate each point with same walking-distance
            # Manipulate current position
            y,x = pos # Unpack the coordinate-point (which is a tuple)
            grid[y][x] = walked # Insert the walkingdistance
            
            # Check possible future positions
            y_down, y_up, x_down, x_up = y-1, y+1, x-1, x+1 # All the possible moves
            # Check case 1 of 4
            if y_down >= 0: # Check for grid-boundraries
                if grid[y_down][x] == GRID_UNITINIALIZED: # Check if we have not been there and can actually go there
                    grid[y_down][x] = walked # Insert the walkingdistance to the new position
                    new_batch.append((y_down,x)) # Add the new point to the list of points-to-inspect
            # Check case 2 of 4
            if x_down >= 0: # Check for grid-boundraries
                if grid[y][x_down] == GRID_UNITINIALIZED: # Check if we have not been there and can actually go there
                    grid[y][x_down] = walked # Insert the walkingdistance to the new position
                    new_batch.append((y,x_down)) # Add the new point to the list of points-to-inspect
            # Check case 3 of 4
            if y_up < GRID_SIZE: # Check for grid-boundraries
                if grid[y_up][x] == GRID_UNITINIALIZED: # Check if we have not been there and can actually go there
                    grid[y_up][x] = walked # Insert the walkingdistance to the new position
                    new_batch.append((y_up,x)) # Add the new point to the list of points-to-inspect
            # Check case 4 of 4
            if x_up < GRID_SIZE: # Check for grid-boundraries
                if grid[y][x_up] == GRID_UNITINIALIZED: # Check if we have not been there and can actually go there
                    grid[y][x_up] = walked # Insert the walkingdistance to the new position
                    new_batch.append((y,x_up)) # Add the new point to the list of points-to-inspect
                    
        walked += 1 # Increase the pointer of walked distance
        if len(new_batch) > 0: # If we have found new points to inspect, append them to keep iterating
            points_to_inspect.append(new_batch)
    
    # For Debugging
    plot_grid(grid,f'Calculated distance-field, depth {depth}')
    
    # Check if the new destination-point can be reached at all, if not back-track one iteration
    p_target = grid[grid_point_from[0]][grid_point_from[1]]
    if p_target == GRID_UNITINIALIZED: # Cant reach the point
        print(f'Target is {grid_point_from}, value of target is {p_target} which is ok.')
        # TODO: Back-Track one step
        # depth -= 1
        print('Got Stuck in a dead-end!')
        return None
    
    # Calculate best path from grid_target to grid_point_from
    new_path = [grid_point_to]
    best_position = grid_point_to
    best_distance = grid[grid_point_to[0]][grid_point_to[1]] # Get the Distance of the point
    while best_distance > 0:
        new_path.append(best_position) # Placed here as to not append the next-startingpoint
        y,x = best_position # Current position
        y_down, y_up, x_down, x_up = y-1, y+1, x-1, x+1 # Allth possible moves
        
        # Check case 1 of 4
        if y_down >= 0:
            p_distance = grid[y_down][x] # Get Distance of inspected point
            if p_distance < best_distance: # If it is closer to the target, choose this point as next point
                best_distance = p_distance
                best_position = (y_down,x)
        
        # Check case 2 of 4
        if x_down >= 0:
            p_distance = grid[y][x_down] # Get Distance of inspected point
            if p_distance < best_distance: # If it is closer to the target, choose this point as next point
                best_distance = p_distance
                best_position = (y,x_down)
                
        # Check case 3 of 4
        if y_up < GRID_SIZE:
            p_distance = grid[y_up][x] # Get Distance of inspected point
            if p_distance < best_distance: # If it is closer to the target, choose this point as next point
                best_distance = p_distance
                best_position = (y_up,x)
                
        # Check case 4 of 4
        if x_up < GRID_SIZE:
            p_distance = grid[y][x_up] # Get Distance of inspected point
            if p_distance < best_distance: # If it is closer to the target, choose this point as next point
                best_distance = p_distance
                best_position = (y,x_up)
    
    # For Debugging
    print_grid = grid.copy()
    for w_pos in new_path:
        y,x = w_pos
        print_grid[y][x] = GRID_PATH
    plot_grid(print_grid,f'Calculated best path, depth {depth}')
    
    # Update all parameters to go into next iteration
    del new_path[0] # Have to remove this one to prevent doubles
    walls += new_path
    route += new_path
    depth += 1
    
    #Check end-game condition
    if depth+1 == len(stations):
        route.append(grid_point_from)
        plot_grid(print_grid,'Found the solution!')
        print(f'Optimal route: {route}')
        print('All done')
        return route   
    
    print('Moving down +1 in depth')
    return solve_grid(stations=stations,walls=walls,route=route,depth=depth)

In [4]:
def draw_final_grid(coords, stations):
    # Generate the empty grid
    grid = [[GRID_UNITINIALIZED for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    
    # Mark the known wall-positions
    for w_pos in coords:
        y,x = w_pos
        grid[y][x] = GRID_WALL
    
    # For Debugging
    plot_grid(grid,'The final grid',stations)

In [5]:
stations = [(1,1),(4,8),(2,3),(0,0),(9,9)]
#walls = [(0,2),(1,2),(2,2),(3,2),(4,2)]
walls = []
walls = solve_grid(stations=stations, walls=walls)


Created grid with walls, depth 0
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	|  	|

Calculated distance-field, depth 0
|0	| 1	| 2	| 3	| 4	| 5	| 6	| 7	| 8	| 9	|
|1	| 2	| 3	| 4	| 5	| 6	| 7	| 8	| 9	| 10	|
|2	| 3	| 4	| 5	| 6	| 7	| 8	| 9	| 10	| 11	|
|3	| 4	| 5	| 6	| 7	| 8	| 9	| 10	| 11	| 12	|
|4	| 5	| 6	| 7	| 8	| 9	| 10	| 11	| 12	| 13	|
|5	| 6	| 7	| 8	| 9	| 10	| 11	| 12	| 13	| 14	|
|6	| 7	| 8	| 9	| 10	| 11	| 12	| 13	| 14	| 15	|
|7	| 8	| 9	| 10	| 11	| 12	| 13	| 14	| 15	| 16	|
|8	| 9	| 10	| 11	| 12	| 13	| 14	| 15	| 16	| 17	|
|9	| 10	| 11	| 12	| 13	| 14	| 15	| 16	| 17	| 18	|

Calculated best path, depth 0
|0	| x	| x	| x	| x	| x	| x	| x	| x

In [6]:
walls

[(9, 9),
 (8, 9),
 (7, 9),
 (6, 9),
 (5, 9),
 (4, 9),
 (3, 9),
 (2, 9),
 (1, 9),
 (0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0),
 (1, 0),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 3),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (3, 8),
 (2, 8),
 (1, 8),
 (1, 7),
 (1, 6),
 (1, 5),
 (1, 4),
 (1, 3),
 (1, 2),
 (1, 1)]

In [7]:
draw_final_grid(walls,stations)


The final grid
|S	| W	| W	| W	| W	| W	| W	| W	| W	| W	|
|W	| S	| W	| W	| W	| W	| W	| W	| W	| W	|
|W	| W	| W	| S	|  	|  	|  	|  	| W	| W	|
| 	|  	|  	| W	|  	|  	|  	|  	| W	| W	|
| 	|  	|  	| W	| W	| W	| W	| W	| S	| W	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	| W	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	| W	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	| W	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	| W	|
| 	|  	|  	|  	|  	|  	|  	|  	|  	| S	|


# TODO: Backtracking does not jet work, also getting stuck in a endless-loop when checking the backtracing-criteria (2019-02-11)